In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import *
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras.layers import BatchNormalization
from keras import initializers
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU

import pandas as pdS
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


Using TensorFlow backend.


In [3]:

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation



In [4]:
train_data = ImageDataGenerator(
     rescale=1 / 255
)#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

#train_data = ImageDataGenerator()

test_datagen = ImageDataGenerator()

train_generator = train_data.flow_from_directory(
    "../NEW2/TRAIN/",
    target_size=(600, 450),
    batch_size=16,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    "../NEW2/TEST",
    target_size=(600, 450),
    batch_size=16,
    class_mode='categorical')

Found 4799 images belonging to 4 classes.
Found 704 images belonging to 4 classes.


In [5]:
model=Model(inputs=base_model.input,outputs=preds)

for layer in model.layers[:-4]:
    layer.trainable=False
    
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])



    
print(model.summary())
step_size_train=train_generator.n//train_generator.batch_size
step_size_validate = validation_generator.n // validation_generator.batch_size

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                  steps_per_epoch=step_size_train,
                   epochs=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

Epoch 1/1
299/299 [==============================] - 3211s 11s/step - loss: 0.9783 - acc: 0.5809


In [6]:
model.evaluate_generator(validation_generator, 3, verbose=1)

3/3 [==============================] - 33s 11s/step


[1.2588147322336833, 0.5208333333333334]

In [27]:

validation_generator.class_indices
train_generator.class_indices

{'BCC': 0, 'BKL': 1, 'MEL': 2, 'NV': 3}

In [30]:
%matplotlib inline
from keras.preprocessing import image
import os
import shutil
 

correct = 0
incorrect = 0


directories = ["BCC", "BKL", "MEL", "NV"]

for directory in directories:
    list_of_images_in_directory = os.listdir("../NEW2/TEST/" + str(directory))
    for picture in list_of_images_in_directory:
        img = image.load_img("../NEW2/TEST/" + str(directory) + "/" + str(picture), target_size=(600, 450))
        x = image.img_to_array(img) / 255.
        
        prediction = model.predict(x.reshape((1, 600, 450, 3)))
        
        print(prediction)
        temp = list(prediction)
        print(temp)
        index_max_value = temp.index(max(temp))
        print(index_max_value)
        
        
        if directory == "BCC":
            if index_max_value == 0:
                correct += 1
            else:
                incorrect += 1
                
        if directory == "BKL":
            if index_max_value == 1:
                correct += 1
            else:
                incorrect += 1
                
        if directory == "MEL":
            if index_max_value == 2:
                correct += 1
            else:
                incorrect += 1
                
                
        if directory == "NV":
            if index_max_value == 3:
                correct += 1
            else:
                incorrect += 1
                
        break
                
        
print(correct / (incorrect + correct))

[[0.15521996 0.13754126 0.29149783 0.415741  ]]
[array([0.15521996, 0.13754126, 0.29149783, 0.415741  ], dtype=float32)]
0
[[0.09174764 0.15840836 0.46754998 0.282294  ]]
[array([0.09174764, 0.15840836, 0.46754998, 0.282294  ], dtype=float32)]
0
[[0.14669251 0.1506827  0.34362063 0.35900408]]
[array([0.14669251, 0.1506827 , 0.34362063, 0.35900408], dtype=float32)]
0
[[0.08507384 0.19680119 0.47484845 0.24327648]]
[array([0.08507384, 0.19680119, 0.47484845, 0.24327648], dtype=float32)]
0
0.25
